*** EXTRACCION DE CARACTETISTICAS LBPH ***

Datos del Drive. 
1. Conexión y permisos con el drive personal

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


2. Ubicación de las carpetas a utilizar

In [15]:
#%cd /content/gdrive/My Drive/0_Desarrollo_Tesis/2.Resultados_Preprocesamiento/
%cd /content/gdrive/My Drive/0_Desarrollo_Tesis/

/content/gdrive/.shortcut-targets-by-id/13aHaot6tbm7SROiS_JzQgfDtjZ_DM4PH/0_Desarrollo_Tesis


3. Listar la lista del contenido de la ruta

In [ ]:
!ls /content/gdrive/My\ Drive/0_Desarrollo_Tesis/*

***FUNCION HISTOGRAMA***


In [4]:
def histogram(imgGRAY):
    #histograma
    hist = cv.calcHist([imgGRAY],[0],None,[256],[0,256])
    # return the histogram
    return hist.flatten()

    
# CODIGO DE PRUEBAS

def funcion_prueba(path_EXTRACCION,path_REDIMENSIONADO,anexo_archivo):
  print("path_EXTRACCION"+path_EXTRACCION)
  print("path_REDIMENSIONADO"+path_REDIMENSIONADO)
  print("anexo_archivo"+anexo_archivo)

*** LBPH ***

In [5]:
def getLBPimage(gray_image):
  ### Step 0: Step 0: Convert an image to grayscale
  gray_image = cv2.cvtColor(gray_image, cv2.COLOR_BGR2GRAY)
  imgLBP = np.zeros_like(gray_image)
  neighboor = 3 
  for ih in range(0,imgLBP.shape[0] - neighboor):
      for iw in range(0,imgLBP.shape[1] - neighboor):
          ### Step 1: 3 by 3 pixel
          img          = gray_image[ih:ih+neighboor,iw:iw+neighboor]
          center       = img[1,1]
          img01        = (img >= center)*1.0
          img01_vector = img01.T.flatten()
          ### Step 2: **Binary operation**:
          img01_vector = np.delete(img01_vector,4)
          ### Step 3: Decimal: Convert the binary operated values to a digit.
          where_img01_vector = np.where(img01_vector)[0]
          if len(where_img01_vector) >= 1:
              num = np.sum(2**where_img01_vector)
          else:
              num = 0
          imgLBP[ih+1,iw+1] = num
  return(imgLBP)

In [6]:
import matplotlib.pyplot as plt 
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

from skimage import io, color
import cv2 as cv


def extraccion_imagen_LBPH(path_EXTRACCION,path_REDIMENSIONADO, anexo_archivo):

  path_redimensionadas=path_REDIMENSIONADO #Imagen redimensionada a color
  path_resultado=path_EXTRACCION  #Imagen Eigen
  os.makedirs(path_resultado, exist_ok=True)

  KA=anexo_archivo.split(sep='/', maxsplit=1)
  K=KA[1].split(sep='_', maxsplit=1)

  #Numero de componentes principales
  k_componentes =int(K[1]) 

  #CREACION DE ARCHIVOS CSV CON LAS CARACTERISTICAS
  raiz_histograma_caracteristicas="5.Resultados_Histogramas/"+anexo_archivo
  os.makedirs(raiz_histograma_caracteristicas, exist_ok=True)
  histograma_caracteristicas=raiz_histograma_caracteristicas+"/histograma_caracteristicas.csv"
  histograma_ruta_redimensionada=raiz_histograma_caracteristicas+"/histograma_ruta_redimensionada.csv"
  histograma_ruta_eigen=raiz_histograma_caracteristicas+"/histograma_ruta_eigen.csv"

  #Guarda la ruta de imagen REDIMENSIONADA
  archivo_ruta_redimensionada= open(histograma_ruta_redimensionada, "w+") 
  #Caracteristicas imagenes LBP a Histogramas
  archivo_histograma= open(histograma_caracteristicas, "w+") 
  #Guarda la ruta de imagen LBPH
  archivo_ruta_LBPH= open(histograma_ruta_LBPH, "w+")


  #For que recorre las carpetas(sujetos) del path
  contador = 0
  for aa in os.listdir(path_redimensionadas):
    #For que recorre la imagen de cada carpeta(sujeto)
    for f in os.listdir(path_redimensionadas + '/' + aa):
      contador = contador + 1
      #aa Carpeta del sujeto 
      #f Nombre de la imagen del sujeto
      #Ruta de la imagen
      imgPath = path_redimensionadas + '/' + aa + '/' + f

      #Guarda la ruta de imagen REDIMENSIONADA
      archivo_ruta_redimensionada.write(imgPath)
      archivo_ruta_redimensionada.write("\n")

      #Abrir Imagen (Lee la imagen en formtado BGR)
      imgBGR=''
      imgBGR = cv2.imread(imgPath)
      '''
      fig = plt.figure(figsize=(20,8))
      ax  = fig.add_subplot(1,3,1)
      ax.imshow(imgBGR)
      '''
      
      #*****************
      #    LBPH
      #*****************
      imgLBP    = getLBPimage(imgBGR)
      #Guarda imagenes imagenes con cuadros
      os.makedirs(path_resultado+ '/'+ aa, exist_ok=True)
      filename=(path_resultado + '/' + aa + '/' + f) 
      print("Filename "+filename)
      cv2.imwrite(filename, imgLBP.reshape(180,180)) 

      #Guarda la ruta de imagen EIGEN
      archivo_ruta_LBPH.write(filename)
      archivo_ruta_LBPH.write("\n")

      #*****************
      #    Histograma
      #*****************
      fv_hist = histogram(imgLBP)
      i=0
      for b in fv_hist:
        i=i+1
        archivo_histograma.write("%.20f" % float(b))
        if(i<=(len(fv_hist)-1)):
          archivo_histograma.write(",")
      archivo_histograma.write("\n")


In [7]:
# Importar Librerias

import dlib, cv2, os
from imutils import face_utils
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage import io, color
import cv2 as cv

def menu_tesis():
  print("----------- Tesis U Catolica -----------")
  print("----------- Seleccione la tecnica a implementar -----------")
  print("1. LPBH")
  print("-----------------------------------")

def menu_opcion():
  print("----------- Medidas de similaridad -----------")
  print("1. Perros")
  print("2. Buscar Perro")
  print("3. Gatos")
  print("4. Buscar Gato")
  print("5. Gatos QUERY")
  print("6. Perros QUERY")
  print("-----------------------------------")

def seleccion_menu():
  menu_tesis()
  mopcion = int(input("Seleccione una opcion: "))
  caso_menu(mopcion)

def seleccion_opcion(mopcion):
  menu_opcion()
  opcion = int(input("Seleccione una opcion: "))
  print(mopcion)
  print(opcion)
  caso_opcion(mopcion,opcion)

In [9]:
#def caso_opcion(self, mopcion,opcion):
class caso_opcion:
  def __init__(self, mopcion,opcion):
    #*****************
    #      LBPH
    #*****************

    if mopcion== 1 and opcion == 1: #(1. LBPH, 1. Perros)
      print('1. LBPH, 1. Perros')
      #path_EXTRACCION = '4.Resultados_ext_caracteristicas/LBPH_Perros' #1
      path_REDIMENSIONADO='2.Resultados_Preprocesamiento/Perros/ROSTRO_REDIMENSIONADO'#2

      #**************************
      #For que recorre las carpetas(sujetos) del path
      contador_carpetas = 0
      contador_imagenes = 0
      for aa in os.listdir(path_REDIMENSIONADO):
        contador_carpetas=contador_carpetas+1
        #For que recorre la imagen de cada carpeta(sujeto)
        for f in os.listdir(path_REDIMENSIONADO + '/' + aa):
          contador_imagenes = contador_imagenes + 1
      
      print("CANTIDAD DE CARPETAS "+ str(contador_carpetas))
      print("CANTIDAD DE IMAGENES "+ str(contador_imagenes))
          
      for i in range(1, (contador_imagenes-3)):  #****CAMBIAR EL VALOR 
        anexo_archivo='LBPH_Perros'+'/K_'+str(i)
        path_EXTRACCION = '4.Resultados_ext_caracteristicas/LBPH_Perros'+'/K_'+str(i)
        extraccion_imagen_LBPH(path_EXTRACCION,path_REDIMENSIONADO, anexo_archivo)
        
      print(os.path.isdir(path_EXTRACCION))
      print(os.path.isdir(path_REDIMENSIONADO))
      
    elif mopcion== 1 and opcion == 2: #(1. LBPH, 2. Buscar Perro)
      print('1. LBPH, 2. Buscar Perro')
      #path_EXTRACCION = '4.Resultados_ext_caracteristicas/LBPH_Perros_Entrada' #1
      path_REDIMENSIONADO='2.Resultados_Preprocesamiento/Buscar_perro/ROSTRO_REDIMENSIONADO'#2

      #**************************
      #For que recorre las carpetas(sujetos) del path
      contador_carpetas = 0
      contador_imagenes = 0
      for aa in os.listdir(path_REDIMENSIONADO):
        contador_carpetas=contador_carpetas+1
        #For que recorre la imagen de cada carpeta(sujeto)
        for f in os.listdir(path_REDIMENSIONADO + '/' + aa):
          contador_imagenes = contador_imagenes + 1
      
      print("CANTIDAD DE CARPETAS "+ str(contador_carpetas))
      print("CANTIDAD DE IMAGENES "+ str(contador_imagenes))
          
      for i in range(1, (contador_imagenes-3)):  #****CAMBIAR EL VALOR 
        anexo_archivo='LBPH_Perros_Entrada'+'/K_'+str(i)
        path_EXTRACCION = '4.Resultados_ext_caracteristicas/LBPH_Perros_Entrada'+'/K_'+str(i)
        extraccion_imagen_eigen(path_EXTRACCION,path_REDIMENSIONADO,anexo_archivo)
        
      print(os.path.isdir(path_EXTRACCION))
      print(os.path.isdir(path_REDIMENSIONADO))
      
    elif mopcion== 1 and opcion == 3:  #(1. LBPH, 3. Gatos)
      print('1. LBPH, 3. Gatos')
      #path_EXTRACCION = '4.Resultados_ext_caracteristicas/LBPH_Gatos' #1
      path_REDIMENSIONADO='2.Resultados_Preprocesamiento/Gatos/ROSTRO_REDIMENSIONADO'#2

      #**************************
      #For que recorre las carpetas(sujetos) del path
      contador_carpetas = 0
      contador_imagenes = 0
      for aa in os.listdir(path_REDIMENSIONADO):
        contador_carpetas=contador_carpetas+1
        #For que recorre la imagen de cada carpeta(sujeto)
        for f in os.listdir(path_REDIMENSIONADO + '/' + aa):
          contador_imagenes = contador_imagenes + 1
      
      print("CANTIDAD DE CARPETAS "+ str(contador_carpetas))
      print("CANTIDAD DE IMAGENES "+ str(contador_imagenes))
          
      for i in range(1, (contador_imagenes-3)):  #****CAMBIAR EL VALOR 
        anexo_archivo='LBPH_Gatos'+'/K_'+str(i)
        path_EXTRACCION = '4.Resultados_ext_caracteristicas/LBPH_Gatos'+'/K_'+str(i)
        extraccion_imagen_eigen(path_EXTRACCION,path_REDIMENSIONADO,anexo_archivo)
        
      print(os.path.isdir(path_EXTRACCION))
      print(os.path.isdir(path_REDIMENSIONADO))

    elif mopcion== 1 and opcion == 4: #(1. LBPH, 4. Buscar Gato)
      print('1. LBPH, 4. Buscar Gato')
      path_EXTRACCION = '4.Resultados_ext_caracteristicas/LBPH_Gatos_Entrada' #1
      path_REDIMENSIONADO='2.Resultados_Preprocesamiento/Buscar_gato/ROSTRO_REDIMENSIONADO'#2

      #**************************
      #For que recorre las carpetas(sujetos) del path
      contador_carpetas = 0
      contador_imagenes = 0
      for aa in os.listdir(path_REDIMENSIONADO):
        contador_carpetas=contador_carpetas+1
        #For que recorre la imagen de cada carpeta(sujeto)
        for f in os.listdir(path_REDIMENSIONADO + '/' + aa):
          contador_imagenes = contador_imagenes + 1
      
      print("CANTIDAD DE CARPETAS "+ str(contador_carpetas))
      print("CANTIDAD DE IMAGENES "+ str(contador_imagenes))
          
      for i in range(1, (contador_imagenes-3)):  #****CAMBIAR EL VALOR 
        anexo_archivo='LBPH_Gatos_Entrada'+'/K_'+str(i)
        path_EXTRACCION = '4.Resultados_ext_caracteristicas/LBPH_Gatos_Entrada'+'/K_'+str(i)
        extraccion_imagen_eigen(path_EXTRACCION,path_REDIMENSIONADO,anexo_archivo)
        
      print(os.path.isdir(path_EXTRACCION))
      print(os.path.isdir(path_REDIMENSIONADO))
      
    elif mopcion== 1 and opcion == 5: #(1. LBPH, 5. Gatos QUERY)
      print('1. LBPH, 5. Gatos QUERY')
      #path_EXTRACCION = '4.Resultados_ext_caracteristicas/LBPH_Gatos_QUERY' #1
      path_REDIMENSIONADO='2.Resultados_Preprocesamiento/Gatos_QUERY/ROSTRO_REDIMENSIONADO'#2 

      #For que recorre las carpetas(sujetos) del path
      contador_carpetas = 0
      contador_imagenes = 0
      for aa in os.listdir(path_REDIMENSIONADO):
        contador_carpetas=contador_carpetas+1
        #For que recorre la imagen de cada carpeta(sujeto)
        for f in os.listdir(path_REDIMENSIONADO + '/' + aa):
          contador_imagenes = contador_imagenes + 1
      
      print("CANTIDAD DE CARPETAS "+ str(contador_carpetas))
      print("CANTIDAD DE IMAGENES "+ str(contador_imagenes))
          
      for i in range(1, (contador_imagenes-3)):
        anexo_archivo='LBPH_Gatos_QUERY'+'/K_'+str(i)
        path_EXTRACCION = '4.Resultados_ext_caracteristicas/LBPH_Gatos_QUERY'+'/K_'+str(i)
        extraccion_imagen_eigen(path_EXTRACCION,path_REDIMENSIONADO,anexo_archivo)

      print(os.path.isdir(path_EXTRACCION))
      print(os.path.isdir(path_REDIMENSIONADO))
      
    elif mopcion== 1 and opcion == 6: #(1. LBPH, 6. Perros QUERY)
      print('1. LBPH, 6. Perros QUERY')
      #path_EXTRACCION = '4.Resultados_ext_caracteristicas/LBPH_Perros_QUERY' #1
      path_REDIMENSIONADO='2.Resultados_Preprocesamiento/Perros_QUERY/ROSTRO_REDIMENSIONADO'#2

      #For que recorre las carpetas(sujetos) del path
      contador_carpetas = 0
      contador_imagenes = 0
      for aa in os.listdir(path_REDIMENSIONADO):
        contador_carpetas=contador_carpetas+1
        #For que recorre la imagen de cada carpeta(sujeto)
        for f in os.listdir(path_REDIMENSIONADO + '/' + aa):
          contador_imagenes = contador_imagenes + 1
      
      print("CANTIDAD DE CARPETAS "+ str(contador_carpetas))
      print("CANTIDAD DE IMAGENES "+ str(contador_imagenes))
          
      for i in range(1, (contador_imagenes-3)):  #****CAMBIAR EL VALOR 
        anexo_archivo='LBPH_Perros_QUERY'+'/K_'+str(i)
        path_EXTRACCION = '4.Resultados_ext_caracteristicas/LBPH_Perros_QUERY'+'/K_'+str(i)
        extraccion_imagen_eigen(path_EXTRACCION,path_REDIMENSIONADO,anexo_archivo)
        
      print(os.path.isdir(path_EXTRACCION))
      print(os.path.isdir(path_REDIMENSIONADO))
      
    else:
      #seleccion_opcion()
      print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
      print('Error: La opción ingresada de BUSQUEDA no esta dentro de las opciones, por favor vuelva a ingresar la opción')
      print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
      print('')
      print('')
      print('')
      seleccion_opcion()
   
   
    

In [10]:
def caso_menu(mopcion):
  if mopcion == 1:
    print('LPBH')
    seleccion_opcion(mopcion)
  else:
    print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
    print('Error: La opción ingresada no esta dentro de las opciones, por favor vuelva a ingresar la opción')
    print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
    print('')
    print('')
    seleccion_menu()

seleccion_menu()

----------- Tesis U Catolica -----------
----------- Seleccione la tecnica a implementar -----------
1. LPBH
-----------------------------------
Seleccione una opcion: 1
LPBH
----------- Medidas de similaridad -----------
1. Perros
2. Buscar Perro
3. Gatos
4. Buscar Gato
5. Gatos QUERY
6. Perros QUERY
-----------------------------------
Seleccione una opcion: 5
1
5
1. LBPH, 5. Gatos QUERY


FileNotFoundError: ignored